# Neurofeedback Basic, OpenBCI and NF-Stim

This approach uses the OpenBCI EEG net, and the Stimulus Laptop to deliver the Stimuli
Everything is wireless. This is the most 'basic' approach to do any (other) kind of neurofeedback experiment. As such it can be expanded to do EEG-NF in the scanner, as well.

## The Phases of the NF Experiment

#### **Calibration**
---------------
- Put on the EEG Net (and check/fudge) the Signals until you are happy with the situation.
- Acquire some Session-Localizer(s)
  - Start the Stimulation
  - Stream/obtain some Data
- Run a few Analyses to determine Spatial Filters
- Validate those Spatial Filters using Real-Time Visualization with/without the Filters in Place
- Run a few Analyses to determina Temporal Filters
- Validate Temporal Filters

#### **Measurement**
---------------
- Run Session
  - Start the Stimulation
  - Stream/save Data
  - Apply Spatial and Temporal Filters
  - Update (send to Stimulation) the Feedback Parameters
  - Keep Track of the Behavioral Triggers/Succeses/Failures/Etc
  
#### **Post Measurement**
--------------------
- Final Calibration? Double/Check Signal (see if it drifted)


  
  

# Start Calibration Measurements
We have two calibrations
 - RestingState
 - EOEC
 
We will work first RestingState, and after that EOEC


### Start up the Stimulus
make sure that the wifi hotspot is started on the stimulus computer  
and that this computer is connected to that wifi hotspot  

In [1]:
import sys
sys.path.append("../../callpyff")   # talk to the stimuli
from callpyff import bcinetwork, bcixml 

In [38]:
IP_STIM_COMPUTER = '10.42.0.1'
bcinet = bcinetwork.BciNetwork(IP_STIM_COMPUTER, bcinetwork.FC_PORT, bcinetwork.GUI_PORT, 'bcixml')

__new__: I found an instance of BciNetwork!!
and it uses your requested port: 12346 == 12346
calling close_srvsocket on the previous instance.
closing...


In [39]:
feedbacks = bcinet.getAvailableFeedbacks()
print(feedbacks)

KeyboardInterrupt: 

In [19]:
bcinet.send_init('EEGfMRILocalizer')

In [20]:
bcinet.get_variables()

{'MONITOR_NSCREENS': 2,
 'MONITOR_WIDTH': 40.0,
 'fontheight': 200,
 'color': [0, 0, 0],
 'MONITOR_DEGS_WIDTHBASE': 45,
 'MONITOR_HEIGHT': 30.0,
 'udp_markers_port': 12344,
 'MONITOR_USEDEGS': False,
 'MONITOR_ALLOWGUI': False,
 'EX_EV_IGNORE_KEYS': ['5', 't'],
 'MONITOR_DISPLAYONSCREEN': 1,
 'EVENT_TRIGLOG': 'log/triggerlog.log',
 'EVENT_sendParallel': True,
 'PORT': 6050,
 'udp_markers_host': '127.0.0.1',
 'MONITOR_DEGS_HEIGHTBASE': 35,
 'EYESCLOSED_TIME': 180.0,
 'EVENT_printToTerminal': True,
 'EYESOPEN_TIME': 180.0,
 'MONITOR_FULLSCR': False,
 'STARTKEYS': ['return', 't', '5'],
 'LPT_TRIGGER_WAIT': 0.005,
 'STARTMRIKEYS': ['return', 't', '5'],
 'MONITOR_DISTANCE': 70.0,
 'EVENT_sendTcpIp': True,
 'LOGFILEBASE': 'efl',
 'MONITOR_GAMMA': 1.0,
 'MONITOR_PIXWIDTH': 1280,
 'EVENT_LPTTrigWaitTime': 0.005,
 'EVENT_destip': '127.0.0.1',
 'MONITOR_FLIPVERTICAL': False,
 'MONITOR_PIXHEIGHT': 1024,
 'EVENT_destport': 6050,
 'EVENT_printToTerminalAllowed': [0, 40],
 'EVENT_LPTAddress': 888,
 

In [21]:
bcinet.play()

### start up the EEG
make sure that the EEG is tested  
that the switch on the USB dongle is at GPOI6  
that the switch on the back of the EEG cap is in 'up' position  
that the latency times is set properly [windows](https://openbci.com/forum/index.php?p=/discussion/198/timing-marks-buffering-delays-etc) or [linux](https://askubuntu.com/questions/696593/reduce-request-latency-on-an-ftdi-ubs-to-rs-232-adapter)  
`echo 1 | sudo tee /sys/bus/usb-serial/devices/ttyUSB0/latency_timer`

Then open a terminal  
cd to ~/nf/openBCI_LSL  
then type:  
`python openbci_lsl.py -- stream`

followed by:

`/start`

In [9]:
# make the connection to the EEG Net (EEG Data)
import pylsl
data_stream=pylsl.resolve_byprop("name", "openbci_eeg", timeout=5.0)
if data_stream:
    data_inlet=pylsl.stream_inlet(data_stream[0])
    stream_info = data_inlet.info()
    stream_Fs = stream_info.nominal_srate()
    stream_xml = stream_info.desc()
    chans_xml = stream_xml.child("channels")
    chan_xml_list = []
    ch = chans_xml.child("channel")
    while ch.name() == "channel":
        chan_xml_list.append(ch)
        ch = ch.next_sibling("channel")
    channel_names = [ch_xml.child_value("label") for ch_xml in chan_xml_list]
    data_inlet_dt = data_inlet.time_correction(timeout=5.0)
else:
    raise Exception('No Data Stream Found - Is your EEG Cap running?')

In [7]:
# make the connection to the Stimulus PC (Markers)
marker_stream=pylsl.resolve_byprop("name", "MarkerStream", timeout=5.0)
if marker_stream:
    marker_inlet=pylsl.stream_inlet(marker_stream[0])
else:
    raise Exception('No Marker Stream Found - Is your Stimulus running?')

Exception: No Marker Stream Found - Is your Stimulus running?

In [51]:
# Collect the data

import threading as th

keep_going = True
def key_capture_thread():
    global keep_going
    input()
    keep_going = False


in_data=[]   
in_markers=[]

import pylsl
streams=pylsl.resolve_streams()

data_inlet=pylsl.stream_inlet(streams[0])
marker_inlet=pylsl.stream_inlet(streams[0])

    
th.Thread(target=key_capture_thread, args=(), name='key_capture_thread', daemon=True).start()
while keep_going:
    pass


#try:
#    while True:
#        pass
#except KeyboardInterrupt:
#    pass


#while time.time() - start_time < 10:
#    time.sleep(0.010)
#    data, times = inlet.pull_chunk()
#    data = np.asarray(data)
#    data_timepoints.append(len(times))

In [52]:
import pylsl

In [68]:
data_inlet.pull_chunk()

([[-171881.609375,
   -172205.28125,
   -172271.28125,
   -172264.78125,
   -173413.9375,
   -172545.828125,
   -173815.234375,
   -172159.484375,
   -172120.171875,
   -171087.609375,
   -172478.328125,
   -171663.203125,
   -171832.8125,
   -171674.296875,
   -171945.984375,
   -172548.765625],
  [-172540.3125,
   -172692.0625,
   -172422.859375,
   -172480.71875,
   -172562.515625,
   -171766.703125,
   -172846.375,
   -171676.03125,
   -171642.484375,
   -171450.21875,
   -173473.828125,
   -172378.703125,
   -171528.34375,
   -172274.53125,
   -172095.21875,
   -172955.609375],
  [-171751.1875,
   -171982.4375,
   -171980.9375,
   -172005.1875,
   -174031.953125,
   -172448.609375,
   -174592.09375,
   -172154.640625,
   -172265.671875,
   -171347.796875,
   -172768.171875,
   -171880.59375,
   -171827.984375,
   -172411.390625,
   -171978.21875,
   -172563.171875],
  [-172201.28125,
   -172263.28125,
   -172398.40625,
   -172536.890625,
   -172444.265625,
   -172237.765625,
   -1